<a href="https://colab.research.google.com/github/databyhuseyn/NLP/blob/main/GPT_2_alt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1

In [16]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import inspect ### NEW

In [17]:
import numpy as np

In [18]:
A = np.random.rand(4, 5, 3, 2)
A = torch.tensor(A)
print(A.shape)
A.transpose(-2, -1).shape

torch.Size([4, 5, 3, 2])


torch.Size([4, 5, 2, 3])

In [19]:
class CausalSelfAttention(nn.Module):

  def __init__(self, config):
    super().__init__()
    assert config.n_embd % config.n_head == 0
    # key, query, value projections for all heads but in a batch
    self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
    # output projection
    self.c_proj = nn.Linear(config.n_embd, config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT = 1
    # regularization
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    # not really a 'bias', more of a mask, but following the OpenAI/HF naming though
    self.register_buffer("bias", torch.tril(torch.ones(config.block_size,
                                                       config.block_size)).view(1, 1, config.block_size,
                                                                                config.block_size))
  def forward(self, x):
    B, T, C = x.size()  # batch size, sequence length, embedding dimensionality (n_embd)
    # calculate query, key, values for all head in batch and move head forward to be the batch
    # nh is 'number of heads', hs is 'head size' and C (number of channels) = nh * hs
    # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh * hs=C=768 channels in the Transformer
    qkv = self.c_attn(x)
    q, k , v = qkv.split(self.n_embd, dim=2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    # attention (materializes the large (T,T) matrix for al the queries and keys)

    ### NEW
    # att = (q @ k.transpose(-2, -1)) * (1.0 / torch.sqrt(torch.tensor(k.size(-1), dtype=torch.float32)))
    # att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
    # att = F.softmax(att, dim=-1)
    # y = att @ v # (B, nh, T, T) x (B, nh, T, hs) --> (B, nh, T, hs)
    y = F.scaled_dot_product_attention(q, k, v, is_causal=True)   ### NEW




    y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
    # output projection
    y = self.c_proj(y)
    return y

In [20]:
class MLP(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
    self.gelu    = nn.GELU(approximate='tanh')
    self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT = 1

  def forward(self, x):
    x = self.c_fc(x)
    x = self.gelu(x)
    x = self.c_proj(x)
    return x

In [21]:
class Block(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = CausalSelfAttention(config)
    self.ln_2 = nn.LayerNorm(config.n_embd)
    self.mlp = MLP(config)

  def forward(self, x):
    x = x + self.attn(self.ln_1(x))
    x = x + self.mlp(self.ln_2(x))
    return x

In [22]:
@dataclass
class GPTConfig:
  block_size: int = 1024 # max sequence length
  vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|>
  n_layer: int = 12
  n_head: int = 12
  n_embd: int = 768

In [23]:
class GPT(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config

    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embedding(config.vocab_size, config.n_embd),
        wpe = nn.Embedding(config.block_size, config.n_embd),
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
        ln_f = nn.LayerNorm(config.n_embd)
    ))
    self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    # weight sharing scheme
    self.transformer.wte.weight = self.lm_head.weight

    # init params
    self.apply(self._init_weights)

  def _init_weights(self,module):
    if isinstance(module, nn.Linear):
      std=0.02
      if hasattr(module, 'NANOGPT_SCALE_INIT'):
        std *= (2 * self.config.n_layer) ** -0.5
      torch.nn.init.normal_(module.weight, mean=0.0,std=std)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0,std=0.02)

  def forward(self, idx, targets=None):
    # idx is of shape (B, T)
    B, T = idx.size()
    assert T <= self.config.block_size, f"Can not forward sequence of length {T}, block size is exceeded"
    # forward the token and position embeddings
    pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
    pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
    tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
    x = tok_emb + pos_emb
    # forward the blocks of the transformer
    for block in self.transformer.h:
      x = block(x)
    # forward the final layernorm and the classifier
    x = self.transformer.ln_f(x)
    logits = self.lm_head(x)  # (B, T, vocab_size)
    loss = None
    if targets is not None:
      loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
    return logits, loss

  @classmethod
  def from_pretrained(cls, model_type):
    """Loads pretrained GPT-2 model weights from huggingface"""
    assert model_type in {'gpt2','gpt2-medium','gpt2-large','gpt2-xl'}
    from transformers import GPT2LMHeadModel
    print("Loading weights from pretrained gpt: %s" % model_type)

    # n_layer, n_head, and n_embd are determined from model_type
    config_args = {
          'gpt2':         dict(n_layer = 12, n_head = 12, n_embd =768),   # 124M params
          'gpt2-medium':  dict(n_layer = 24, n_head = 16, n_embd =1024),  # 350M params
          'gpt2-large':   dict(n_layer = 36, n_head = 20, n_embd =1280),  # 774M params
          'gpt2-xl':      dict(n_layer = 48, n_head = 25, n_embd =1600),  # 1558M params
      }[model_type]
    config_args['vocab_size']=50257 # always 50257 for GPT model checkpoints
    config_args['block_size']=1024 # always 1024 for GPT model checkpoints
    # create a from-scratch initializerd minGPT model
    config = GPTConfig(**config_args)
    model = GPT(config)
    sd = model.state_dict()
    sd_keys = sd.keys()
    sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]  # discard this mask / buffer_reg

    # init a huggingface/transformers model
    model_hf = GPT2LMHeadModel.from_pretrained(model_type)
    sd_hf = model_hf.state_dict()

    # copy while ensuring all of the parameters a re aligned and match in names and shapes
    sd_keys_hf = sd_hf.keys()
    sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
    sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]
    transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
    # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla
    # this means that we have to transpose these weights when we import them
    assert len(sd_keys_hf) == len(sd_keys), f'mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}'
    for k in sd_keys_hf:
      if any(k.endswith(w) for w in transposed):
        assert sd_hf[k].shape[::-1] == sd[k].shape
        with torch.no_grad():
          sd[k].copy_(sd_hf[k].t())
      else:
        assert sd_hf[k].shape == sd[k].shape
        with torch.no_grad():
          sd[k].copy_(sd_hf[k])

    return model

  ### NEW
  def configure_optimizers(self, weight_decay, learning_rate, device):
    # start with all of the candidate parameters (that require grad)
    param_dict = {pn: p for pn, p in self.named_parameters()}
    param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
    # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
    # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    num_decay_params = sum(p.numel() for p in decay_params)
    num_nodecay_params = sum(p.numel() for p in nodecay_params)
    print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} params")
    print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} params")
    # Create AdamW optimizer and use the fused version if it is available
    fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused = fused_available and 'cuda' in device
    print(f'using fused AdamW: {use_fused}')
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
    return optimizer

In [24]:
# %%capture results
model = GPT.from_pretrained('gpt2')
print("YAAAYY DIDN'T CRASH")

Loading weights from pretrained gpt: gpt2
YAAAYY DIDN'T CRASH


In [25]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-20 07:27:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2025-09-20 07:27:05 (32.4 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [26]:
with open('input.txt', 'r') as f:
  text = f.read()

data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [27]:

import tiktoken

class DataLoaderLite:
  def __init__(self, B, T):
    self.B = B
    self.T = T

    # at init load tokens from disk and store them in memory
    with open('input.txt', 'r') as f:
      text = f.read()
    enc = tiktoken.get_encoding('gpt2')
    tokens = enc.encode(text)
    self.tokens = torch.tensor(tokens)
    print(f'loaded {len(self.tokens)} tokens')
    print(f'1 epoch = {len(self.tokens) // (B * T)} batches')

    self.current_position = 0

  def next_batch(self):
    B, T = self.B, self.T
    buf = self.tokens[self.current_position : self.current_position + B*T + 1]
    x = (buf[:-1]).view(B, T)
    y = (buf[1:]).view(B, T)
    # advance the position in the tensor
    self.current_position += B * T
    # if loading the next batch would be out of bounds, reset
    if self.current_position + (B * T + 1) > len(self.tokens):
      self.current_position = 0
    return x, y

In [28]:

device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
elif hasattr(torch.backends,'mps') and torch.backends.mps.torch.cuda.is_available():
  device = 'mps'
print(f'using device: {device}')



using device: cuda


In [29]:
# device = 'cpu'
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# with open('input.txt', 'r') as f:
#   text = f.read()
# data = text[:1000]
# tokens = enc.encode(text)
# B, T = 4, 32
# buf = torch.tensor(tokens[:B*T + 1])
# buf = buf.to(device)
# x = buf[:-1].view(B, T)
# y = buf[1:].view(B, T)

In [30]:
torch.manual_seed(1337)
if torch.cuda.is_available():
  torch.cuda.manual_seed(1337)

In [31]:
# B16 / T 1024
train_loader = DataLoaderLite(B = 4, T = 32)

loaded 338025 tokens
1 epoch = 2640 batches


In [32]:
torch.set_float32_matmul_precision('high')

In [33]:
import time

In [34]:
model = GPT(GPTConfig())
model.to(device)
# logits, loss = model(x, y)
# print(loss)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)   ### NEW BETAS and EPS
for i in range(50):
  t0 = time.time()
  x, y = train_loader.next_batch()    # later
  x, y = x.to(device), y.to(device)   # later
  optimizer.zero_grad()
  logits, loss = model(x, y)
  # import code; code.interact(local=locals())
  loss.backward()
  norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1)    ### NEW
  optimizer.step()
  # torch.cuda.synchronize()     # gpu stops
  t1 = time.time()
  dt = (t1 - t0) * 1000    # time difference in milliseconds
  tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
  print(f'step {i}, loss: {loss.item()}, dt: {dt:.2f}ms, tok/sec: {tokens_per_sec:.2f}')

step 0, loss: 10.960028648376465, dt: 1397.85ms, tok/sec: 91.57
step 1, loss: 9.687808990478516, dt: 62.87ms, tok/sec: 2035.83
step 2, loss: 9.266006469726562, dt: 90.98ms, tok/sec: 1406.88
step 3, loss: 9.146829605102539, dt: 89.65ms, tok/sec: 1427.79
step 4, loss: 8.658792495727539, dt: 65.93ms, tok/sec: 1941.40
step 5, loss: 8.359376907348633, dt: 35.41ms, tok/sec: 3615.15
step 6, loss: 8.897807121276855, dt: 33.28ms, tok/sec: 3846.22
step 7, loss: 8.86855411529541, dt: 44.18ms, tok/sec: 2897.04
step 8, loss: 8.11693000793457, dt: 32.47ms, tok/sec: 3941.99
step 9, loss: 7.999248504638672, dt: 33.13ms, tok/sec: 3863.85
step 10, loss: 8.325180053710938, dt: 45.21ms, tok/sec: 2831.52
step 11, loss: 7.360818862915039, dt: 35.97ms, tok/sec: 3558.43
step 12, loss: 7.7298712730407715, dt: 33.70ms, tok/sec: 3798.49
step 13, loss: 7.399248123168945, dt: 72.16ms, tok/sec: 1773.92
step 14, loss: 7.489315032958984, dt: 35.98ms, tok/sec: 3557.72
step 15, loss: 7.342123508453369, dt: 37.60ms, tok

In [35]:
### NEW
max_lr = 3e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50
def get_lr(it):
  # 1) linear warmup for warmup_iters steps
  if it<warmup_steps:
    return max_lr * (it+1) / warmup_steps
  # 2) if it > lr_decay_ites, return min learning rate
  if it > max_steps:
    return min_lr
  # 3) in between, use cosine decay down to min learning rate
  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))   # coeff starts at 1 and goes to 0
  return min_lr + coeff * (max_lr - min_lr)

In [36]:
model = GPT(GPTConfig(vocab_size=50304))  ### NEW  from vocab_size=50257
model.to(device)
model = torch.compile(model)     ### NEW
# logits, loss = model(x, y)
# print(loss)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)   ### NEW BETAS and EPS
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate = 6e-4, device = device)
for step in range(max_steps):   ### NEW
  t0 = time.time()
  x, y = train_loader.next_batch()
  x, y = x.to(device), y.to(device)
  optimizer.zero_grad()
  with torch.autocast(device_type=device, dtype=torch.bfloat16):
    logits, loss = model(x, y)
  # import code; code.interact(local=locals())
  loss.backward()
  norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)    ### NEW
  # determine and set the learning rate for this iterarion
  lr = get_lr(step)
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr
  optimizer.step()
  # torch.cuda.synchronize()     # gpu stops
  t1 = time.time()
  dt = (t1 - t0) * 1000    # time difference in milliseconds
  tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
  print(f'step {step:4d} | loss: {loss.item():.6f} | lr {lr:.8f} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}')

num decayed parameter tensors: 50, with 124,354,560 params
num non-decayed parameter tensors: 98, with 121,344 params
using fused AdamW: True


W0920 07:27:50.997000 835 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:2509: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:2509: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:2509: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:2509: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


step    0 | loss: 10.964844 | lr 0.00003000 | norm: 35.1516 | dt: 25245.20ms | tok/sec: 5.07
step    1 | loss: 10.047119 | lr 0.00006000 | norm: 22.3055 | dt: 43.06ms | tok/sec: 2972.61
step    2 | loss: 8.657715 | lr 0.00009000 | norm: 15.1321 | dt: 53.00ms | tok/sec: 2415.29
step    3 | loss: 9.625763 | lr 0.00012000 | norm: 26.5920 | dt: 47.51ms | tok/sec: 2694.17
step    4 | loss: 9.312622 | lr 0.00015000 | norm: 8.4280 | dt: 61.68ms | tok/sec: 2075.15
step    5 | loss: 8.981445 | lr 0.00018000 | norm: 4.9800 | dt: 50.33ms | tok/sec: 2543.45
step    6 | loss: 8.897705 | lr 0.00021000 | norm: 3.7241 | dt: 42.39ms | tok/sec: 3019.84
step    7 | loss: 8.881714 | lr 0.00024000 | norm: 4.0055 | dt: 52.57ms | tok/sec: 2434.69
step    8 | loss: 8.471924 | lr 0.00027000 | norm: 3.6931 | dt: 49.75ms | tok/sec: 2572.79
step    9 | loss: 8.695068 | lr 0.00030000 | norm: 3.1325 | dt: 43.04ms | tok/sec: 2973.79
step   10 | loss: 8.115356 | lr 0.00030000 | norm: 3.4176 | dt: 43.63ms | tok/sec: 2

In [37]:
logits.dtype

torch.bfloat16

In [38]:
model.transformer.wte.weight.dtype

torch.float32

In [39]:
# Worst case scenario
-math.log(1/50257)

10.82490511970208

In [40]:
num_return_sequences = 5
max_length = 30

In [41]:
# model = GPT.from_pretrained('gpt2')
# model.eval()
# model.to(device)

In [42]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a data scientist,")
tokens = torch.tensor(tokens, dtype = torch.long) # (8, )
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) #(5, 8)
x = tokens.to('cuda')

In [43]:
import torch
# generate! right now x is (B, T) where B = 5, T = 8
# set the seed to 42
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
  # forward the model to get the logits
  with torch.no_grad():
    logits, _ = model((x))  # (B, T, vocab_size)
    # take the logits at the last position
    logits = logits[:, -1, :]
    # get the porbabilities
    probs = F.softmax(logits, dim=-1)
    # do top-k sampling of 50 (huggingface pielien default)
    # topk_probs here becomes (5, 50), topk_indices is (5, 50)
    topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
    # select a token from the top-k probabilities
    ix = torch.multinomial(topk_probs, 1) # (B, 1)
    # gather the corresponding indices
    xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
    # append to the sequence
    x = torch.cat((x, xcol), dim=1)

In [44]:
for i in range(num_return_sequences):
  tokens = x[i, :max_length].tolist()
  decoded = enc.decode(tokens)
  print(">", decoded)

> Hello, I'm a data scientist,

 my I
I ofI have? myThe the; all of, the as a.

> Hello, I'm a data scientist, have not it
 you not the good
, it not him you we
..
 you;

> Hello, I'm a data scientist,.
:

 tocius', and', ofUS.







> Hello, I'm a data scientist,
 is'd

 a:
 my- a
cius thatUS the me
US do toIN
> Hello, I'm a data scientist, the to, and
 not my?:
 with, be,'US
 you;
..




---



# PART 2

In [45]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


In [46]:
import torch
buf = torch.tensor(tokens[:24 + 1])
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x)
print(y)


tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [ ]:
# Create DataLoader